<a href="https://colab.research.google.com/github/veda-sunkara/StreetToCloud/blob/master/FCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
LR = 1e-3
EPOCHS = 1000
EPOCHS_PER_UPDATE = 1
RUNNAME = "1e3_flood_0"

In [2]:
from google.colab import auth
auth.authenticate_user()

!curl https://sdk.cloud.google.com | bash

!gcloud init

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   443  100   443    0     0  17720      0 --:--:-- --:--:-- --:--:-- 17720
######################################################################## 100.0%
Running install script from: /tmp/tmp.BuKbpPQB1S/install_google_cloud_sdk.bash
which curl
curl -# -f https://dl.google.com/dl/cloudsdk/channels/rapid/google-cloud-sdk.tar.gz
######################################################################## 100.0%

mkdir -p /root
"/root/google-cloud-sdk" already exists and may contain out of date files.
Remove /root/google-cloud-sdk or select a new installation directory, then run again.
Welcome! This command will take you through the configuration of gcloud.

Settings from your current configuration [default] are:
component_manager:
  disable_update_check: 'True'
compute:
  gce_metadata_read_timeout_sec: '0'
core:
  account: veda@clou

In [3]:
!cd /home
!pwd

/content


In [4]:
import torch
from torchvision import transforms
import torchvision.transforms.functional as F
import random

class InMemoryDataset(torch.utils.data.Dataset):
  
  def __init__(self, data_list, preprocess_func):
    self.data_list = data_list
    self.preprocess_func = preprocess_func
  
  def __getitem__(self, i):
    return self.preprocess_func(self.data_list[i])
  
  def __len__(self):
    return len(self.data_list)


def processAndAugment(data):
  (x,y) = data
  im,label = x.copy(), y.copy()

  # convert to PIL for easier transforms
  im1 = Image.fromarray(im[0])
  im2 = Image.fromarray(im[1])
  label = Image.fromarray(label.squeeze())

  # Get params for random transforms
  i, j, h, w = transforms.RandomCrop.get_params(im1, (256, 256))
  
  im1 = F.crop(im1, i, j, h, w)
  im2 = F.crop(im2, i, j, h, w)
  label = F.crop(label, i, j, h, w)
  if random.random() > 0.5:
    im1 = F.hflip(im1)
    im2 = F.hflip(im2)
    label = F.hflip(label)
  if random.random() > 0.5:
    im1 = F.vflip(im1)
    im2 = F.vflip(im2)
    label = F.vflip(label)
  
  norm = transforms.Normalize([0.6851, 0.5235], [0.0820, 0.1102])
  im = torch.stack([transforms.ToTensor()(im1).squeeze(), transforms.ToTensor()(im2).squeeze()])
  im = norm(im)
  label = transforms.ToTensor()(label).squeeze()
  if torch.sum(label.gt(.003) * label.lt(.004)):
    label *= 255
  label = label.round()

  return im, label


def processTestIm(data):
  (x,y) = data
  im,label = x.copy(), y.copy()
  norm = transforms.Normalize([0.6851, 0.5235], [0.0820, 0.1102])
  #label[0][0][0] = 255
  
  # convert to PIL for easier transforms
  im_c1 = Image.fromarray(im[0]).resize((512,512))
  im_c2 = Image.fromarray(im[1]).resize((512,512))
  label = Image.fromarray(label.squeeze()).resize((512,512))

  im_c1s = [F.crop(im_c1, 0, 0, 256, 256), F.crop(im_c1, 0, 256, 256, 256),
            F.crop(im_c1, 256, 0, 256, 256), F.crop(im_c1, 256, 256, 256, 256)]
  im_c2s = [F.crop(im_c2, 0, 0, 256, 256), F.crop(im_c2, 0, 256, 256, 256),
            F.crop(im_c2, 256, 0, 256, 256), F.crop(im_c2, 256, 256, 256, 256)]
  labels = [F.crop(label, 0, 0, 256, 256), F.crop(label, 0, 256, 256, 256),
            F.crop(label, 256, 0, 256, 256), F.crop(label, 256, 256, 256, 256)]


  ims = [torch.stack((transforms.ToTensor()(x).squeeze(),
                    transforms.ToTensor()(y).squeeze()))
                    for (x,y) in zip(im_c1s, im_c2s)]
  ims = [norm(im) for im in ims]
  ims = torch.stack(ims)
  labels = [(transforms.ToTensor()(label).squeeze()) for label in labels]
  labels = torch.stack(labels)
  if torch.sum(labels.gt(.003) * labels.lt(.004)):
    labels *= 255
  labels = labels.round()
  return ims, labels

In [5]:
!pip install rasterio

!gsutil cp gs://cnn_chips/flood_test_data.csv .
!gsutil cp gs://cnn_chips/flood_train_data.csv .
!gsutil cp gs://cnn_chips/flood_valid_data.csv .

Copying gs://cnn_chips/flood_test_data.csv...
/ [1 files][  4.5 KiB/  4.5 KiB]                                                
Operation completed over 1 objects/4.5 KiB.                                      
Copying gs://cnn_chips/flood_train_data.csv...
/ [1 files][ 12.5 KiB/ 12.5 KiB]                                                
Operation completed over 1 objects/12.5 KiB.                                     
Copying gs://cnn_chips/flood_valid_data.csv...
/ [1 files][  4.4 KiB/  4.4 KiB]                                                
Operation completed over 1 objects/4.4 KiB.                                      


In [6]:
import csv
from PIL import Image
import rasterio
import numpy as np
import os
BASEDIR = ''

def getArr(fname):
  return rasterio.open('/' + fname).read()


def download_perm_water_data_from_file(fname):
  with open(fname) as f:
    data_fnames = [tuple(line) for line in csv.reader(f)]
  i = 0
  data = []
  for (x,y) in data_fnames:
    arr_x, arr_y = getArr(x), getArr(y)
    if np.sum((arr_x != arr_x)) == 0:
      ignore = (arr_y == -1)
      ignore = ((np.uint8(ignore) * -1) * 256) + 1
      arr_y *= ignore
      data.append((arr_x, arr_y))
      i+=1
      print(i)
    else:
      print("skipping nan")
  return data

def download_perm_train_data():
  TRAINING_DATA_FILE = BASEDIR + 'flood_train_data.csv'
  return download_perm_water_data_from_file(TRAINING_DATA_FILE)

def download_perm_valid_data():
  VALID_DATA_FILE = BASEDIR + 'flood_valid_data.csv'
  return download_perm_water_data_from_file(VALID_DATA_FILE)

def download_perm_test_data():
  TEST_DATA_FILE = BASEDIR + 'flood_test_data.csv'
  return download_perm_water_data_from_file(TEST_DATA_FILE)

In [7]:
!mkdir /S1
!mkdir /QC_v2

mkdir: cannot create directory ‘/S1’: File exists


In [8]:
!gsutil -m rsync gs://cnn_chips/S1 /S1/
!gsutil -m rsync gs://cnn_chips/QC_v2 /QC_v2/

Building synchronization state...
Starting synchronization...
Building synchronization state...
Starting synchronization...
Copying gs://cnn_chips/QC_v2/Bolivia_233925_QC.tif...
Copying gs://cnn_chips/QC_v2/Bolivia_129334_QC.tif...
Copying gs://cnn_chips/QC_v2/Bolivia_23014_QC.tif...
Copying gs://cnn_chips/QC_v2/Bolivia_195474_QC.tif...
Copying gs://cnn_chips/QC_v2/Bolivia_312675_QC.tif...
Copying gs://cnn_chips/QC_v2/Bolivia_314919_QC.tif...
Copying gs://cnn_chips/QC_v2/Bolivia_290290_QC.tif...
Copying gs://cnn_chips/QC_v2/Bolivia_294583_QC.tif...
Copying gs://cnn_chips/QC_v2/Bolivia_242570_QC.tif...
Copying gs://cnn_chips/QC_v2/Bolivia_103757_QC.tif...
Copying gs://cnn_chips/QC_v2/Bolivia_360519_QC.tif...
Copying gs://cnn_chips/QC_v2/Bolivia_379434_QC.tif...
Copying gs://cnn_chips/QC_v2/Bolivia_432776_QC.tif...
Copying gs://cnn_chips/QC_v2/Bolivia_60373_QC.tif...
Copying gs://cnn_chips/QC_v2/Bolivia_76104_QC.tif...
Copying gs://cnn_chips/QC_v2/Ghana_103272_QC.tif...
Copying gs://cnn_

In [9]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse

# !sudo mkdir files4
# !gcsfuse --implicit-dirs cnn_chips files4

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   653  100   653    0     0  31095      0 --:--:-- --:--:-- --:--:-- 32650
OK
23 packages can be upgraded. Run 'apt list --upgradable' to see them.
gcsfuse is already the newest version (0.30.0).
0 upgraded, 0 newly installed, 0 to remove and 23 not upgraded.


In [10]:
from time import time

def getArrFlood(fname):
  return rasterio.open(fname).read()

def download_flood_water_data_from_list(l):
  i= 0
  tot_nan = 0
  tot_good = 0
  flood_data = []
  for (im_fname, mask_fname) in l:
    if not os.path.exists(os.path.join("", im_fname)):
      print(os.path.join("", im_fname))
      continue
    arr_x = np.nan_to_num(getArrFlood(os.path.join("", im_fname)))
    arr_y = getArrFlood(os.path.join("", mask_fname))
    ignore = (arr_y == -1)
    ignore = ((np.uint8(ignore) * -1) * 256) + 1
    arr_y *= ignore
    arr_y = np.uint8(getArrFlood(os.path.join("", mask_fname)))
    if np.sum((arr_y != arr_y)) == 0:
      arr_x = np.clip(arr_x, -50, 1)
      arr_x = (arr_x + 50) / 51
      if i % 100 == 0:
        print(i)
        print(im_fname, mask_fname)
      i += 1
      flood_data.append((arr_x,arr_y))
    else:
      print("skipping nan")
  print(i)
  return flood_data

def load_flood_train_data():
  basedir = ""
  fname = "flood_train_data.csv"
  with open(fname) as f:
    fname = [tuple(line) for line in csv.reader(f)]
  return download_flood_water_data_from_list(fname)

def load_weak_flood_train_data():
  basedir = ""
  files = [(os.path.join("S1_NoQC", x[1]), os.path.join("NoQC", x[0])) for x in zip(sorted(os.listdir("files3/NoQC")), sorted(os.listdir("files3/S1_NoQC")))]
  files = [x for x in files if "Bolivia" not in x[0]]
  print(files[0:10])
  return download_flood_water_data_from_list(files)

def load_flood_valid_data():
  basedir = ""
  fname = "flood_valid_data.csv"
  with open(fname) as f:
    fname = [tuple(line) for line in csv.reader(f)]
  print(fname, "files!")
  return download_flood_water_data_from_list(fname)

def load_flood_test_data():
  basedir = ""
  fname = "flood_test_data.csv"
  with open(fname) as f:
    fname = [tuple(line) for line in csv.reader(f)]
  return download_flood_water_data_from_list(fname)

def load_flood_bolivia_test_data():
  basedir = ""
  fname = "flood_bolivia_data.csv"
  with open(fname) as f:
    fname = [tuple(line) for line in csv.reader(f)]
  return download_flood_water_data_from_list(fname)

In [11]:
!pwd
!ls
!cd files4
!pwd

/content
adc.json  flood_test_data.csv	flood_valid_data.csv
files4	  flood_train_data.csv	sample_data
/content


In [12]:
train_data = download_perm_train_data()
train_dataset = InMemoryDataset(train_data, processAndAugment)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True, sampler=None,
                  batch_sampler=None, num_workers=0, collate_fn=None,
                  pin_memory=True, drop_last=False, timeout=0,
                  worker_init_fn=None)
train_iter = iter(train_loader)

1
2
3
4
5
skipping nan
6
7
8
9
10
11
12
13
14
skipping nan
15
skipping nan
16
17
18
19
20
21
22
skipping nan
23
24
25
26
skipping nan
27
28
29
skipping nan
30
31
32
33
34
skipping nan
35
36
37
38
39
40
skipping nan
41
42
43
44
skipping nan
45
46
47
48
49
50
51
52
53
54
skipping nan
55
56
57
58
59
60
61
62
63
skipping nan
64
65
66
skipping nan
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
skipping nan
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
skipping nan
136
137
138
139
140
141
142
143
144
145
146
147
148
skipping nan
149
150
151
152
skipping nan
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
skipping nan
skipping nan
203
204
205
skipping nan
206
207
208
skipping nan
209
skipping nan

In [13]:
mkdir checkpoints